In [1]:
import docx
import re
import numpy as np
import pandas as pd
import pyexcel as pe
import os
import numpy as np
import random
import math

#### import data

In [2]:
records = pe.get_records(file_name="Cancer deident data/Deident_GOC_cancer_6.7.16.xlsx")
Id = []
y=[]
Y=[]
for record in records:
    #print("%d : %s" % (record['UNoique ID'], record['Goals of Care Discussed']))
    Id.append(str(record['UNoique ID']))
    Y.append(int(record['Goals of Care Discussed']=='Yes'))
    y.append(str(int(record['Goals of Care Discussed']=='Yes')))
    
data = pd.concat([pd.Series(Id,name='Id'), pd.Series(y,name='y')], axis=1)
data['file'] = 'UID '+data['Id'] + '.docx'

In [3]:
data.tail(2)

,Id,y,file
62,63,1,UID 63.docx
63,64,1,UID 64.docx


In [4]:
def folder_list(path):    
    review = []
    for infile in filelist:
        file = os.path.join(path,infile)
        r = read_data(file)
        review.append(r)
    return review

def read_data(file):
    f = getText(file)
    f = re.sub(r"\[[^\[\]]+\]", "", f)
    return f

def getText(filename):
    doc = docx.Document(filename)
    fullText = []
    for para in doc.paragraphs:
        fullText.append(para.text)
    return '\n'.join(fullText)

In [5]:
filelist = data['file']
path = 'Cancer deident data'
notes = folder_list(path)

#### save dataset

In [7]:
review = pd.Series(notes)
Y = pd.Series(Y)

In [12]:
import pickle
pickle.dump( review, open( "notes.p", "wb" ) )
pickle.dump( Y, open("y.p","wb"))

In [9]:
from sklearn.cross_validation import train_test_split
X_train, X_test, Y_train, Y_test = train_test_split(notes, Y, train_size=.80)

In [10]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer

binary_vectorizer = CountVectorizer(stop_words = "english",binary=True, ngram_range=(1,2)).fit(notes)
X_train_binary = binary_vectorizer.transform(X_train)
X_test_binary = binary_vectorizer.transform(X_test)
X_binary = binary_vectorizer.transform(review)

tfidf_vectorizer = TfidfVectorizer(stop_words = "english", ngram_range=(1,2)).fit(notes)
X_train_tfidf = tfidf_vectorizer.transform(X_train)
X_test_tfidf = tfidf_vectorizer.transform(X_test)
X_tfidf = tfidf_vectorizer.transform(review)

In [11]:
from sklearn.datasets import dump_svmlight_file

dump_svmlight_file(X_tfidf,Y,'data_tfidf.dat',zero_based=False)
dump_svmlight_file(X_binary,Y,'data_binary.dat',zero_based=False)